In [8]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torchvision

import torch.nn.functional as F
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

import time
plt.ion()   # interactive mode

from database_functions.databasereader import DatabaseReader

# Custom Functions
from shoe_dataset import ShoeDataSet
from shoe_dataset import train_transformations, valid_transformations
from feature_maps_extractor import ExtractFeatureMaps
from class_activation_maps import ScoreCam

torch.manual_seed(1994)

ModuleNotFoundError: No module named 'database_functions'

In [2]:
class custom_model:
    
    def __init__(self, model_type,unfreeze_layers, freeze_factor = 1 ,pretrained = True, output_dir = "./runs/"):
        
        if torch.cuda.is_available():      
            self.device = torch.device("cuda:0")
            print("Model set up on the GPU")
            
        
        else:
            self.device = torch.device("cpu")
            print("Running on the CPU")
        
        
        # Define the backbone model
        if model_type == "resnet101":
            self.model = models.resnet101(pretrained=pretrained)
        
        elif model_type == "resnet50":
            self.model = models.resnet50(pretrained=pretrained)        
        
        else: 
            print("model type unrecognised")
            
        print(f"Model backbone set to: {model_type}")
        
        # Push the model to device
        self.model = self.model.to(self.device)

        self.unfreeze_layers = unfreeze_layers
        self.freeze_factor = freeze_factor
        
        if self.freeze_factor == 1:
            for name,child in self.model.named_children():
                if name not in self.unfreeze_layers:
                    for param in child.parameters():
                        param.requires_grad = False
        
        else:
            for name,child in self.model.named_children():
                if name not in self.unfreeze_layers:
                    for param in child.parameters():
                        param.requires_gra = False
                    
                elif name in self.unfreeze_layers:    
                    for param in list(child.parameters())[int(self.freeze_factor*len(list(child.parameters()))):]:
                        param.requires_grad = False

                        
        # Defines the output dir for the model to be saved
        self.output_dir = output_dir
        
        from torch.utils.tensorboard import SummaryWriter
        self.writer = SummaryWriter(self.output_dir,)
            
    
    def train(self, loss_func, optimizer, lr_scheduler,learning_rate, epochs, trainloader, valloader, eval_period = 100 ):

        self.trainloader = trainloader
        self.valloader = valloader
        self.learning_rate = learning_rate
        self.eval_period = eval_period
        self.epochs = epochs
        
        
        # Params of the 4th layer to confirm later 
        # that layers are correctly frozen
        self._original_weight = list(self.model.layer4.parameters())[0]
        
        
        print(f"Eval Period set to: {self.eval_period}")

        self.criterion = loss_func()
        self.optimizer = optimizer(filter(lambda p: p.requires_grad, self.model.parameters()),lr = self.learning_rate)
        self.lr_scheduler = lr_scheduler(self.optimizer)

        train_epoch_loss = []
        train_epoch_acc  = []

        val_epoch_loss = []
        val_epoch_acc = []

        best_val_acc = 0.0
        running_training_loss = 0.0

        for epoch in range(self.epochs):
            
            # check for first 3 epochs whether the layers
            # are actually frozen and the weights do not change
            
            if 0 < epoch < 3:
                
                print(f"Did the Weights change in epoch: {epoch}?")
                if list(self.model.layer4.parameters())[0].cpu().numpy().all() == self._original_weight[0].cpu().numpy().all():
                    print("No")
                else:
                    print("Yes, abort!!!")
            
            

            train_batch_loss = []
            train_batch_acc = []

            val_batch_loss = []
            val_batch_acc = []

            # train loop
            for idx, data in enumerate(self.trainloader):

                self.model.train()

                inputs,labels = data["image"],data["label"]

                # Place tensors on GPU
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                # Zero out the accumulated gradients
                self.optimizer.zero_grad()

                outputs = self.model(inputs)

                loss = self.criterion(outputs,labels.float())
                
                running_training_loss+=loss.item()

                #append the mean train loss (woking on a batch)
                #use item() to detach from GPU
                train_batch_loss.append(loss.item())

                # Identify Correct predictions
                correct_preds = [torch.argmax(i)==torch.argmax(j) for i,j in zip(outputs,labels)]
                train_acc = correct_preds.count(True)/len(correct_preds)
                train_batch_acc.append(train_acc)


                # Backwards pass
                loss.backward()


                self.optimizer.step()                                                                
                
                # print out stats every N iterations
                if idx%500 == 0:
                    
                    current_lr = self.optimizer.param_groups[0]['lr']
                    print(f"Current lr: {current_lr}")
                    print(f"Step: {idx}/{len(trainloader)}; Epoch: {epoch+1}/{self.epochs}; Train Batch Loss: {loss.item()}")
            
            
            self.writer.add_scalar("Training Loss", running_training_loss/len(self.trainloader), epoch)
            running_training_loss = 0.0
            
            train_epoch_loss.append(torch.tensor(train_batch_loss).mean())
            train_epoch_acc.append(torch.tensor(train_batch_acc).mean())

            running_val_loss = 0
            print("epoch",epoch)
            print("epoch%eval_per", epoch%self.eval_period)
            # Validation loop every self.eval_period epochs
            eval_count = 0
            if epoch%self.eval_period == 0:
                print(f"Starting evaluating at epoch: {epoch}")
                with torch.no_grad():
                    for idx,data in enumerate(self.valloader):

                        # set model in eval() mode
                        self.model.eval()

                        inputs,labels, self.val_file_name = data["image"], data["label"], data["file_name"]
                        # Place inputs/lables on the GPU
                        inputs = inputs.to(self.device)
                        labels = labels.to(self.device)

                        # Predict outputs
                        outputs = self.model(inputs)

                        # Obtain and append val_batch_loss
                        val_loss = self.criterion(outputs,labels)
                        val_batch_loss.append(loss.item())
                        running_val_loss += val_loss

                        # Obtain and append val_batch_acc
                        correct_preds = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs,labels)]

                        val_acc = correct_preds.count(True)/len(correct_preds)
                        val_batch_acc.append(val_acc)
                        
                        
                        
                        if eval_count%3 == 0:
                            
                            all_feature_maps = []
                            
                            for name in data["file_name"][:2]:
                                all_feature_maps.append(ExtractFeatureMaps(self.model, name))
                                
                            all_feature_maps_stacked = np.vstack(all_feature_maps)
                            self.writer.add_image("Feature Maps of every 10th conv layer", all_feature_maps_stacked,global_step=epoch)
                                
                            
                            all_layers_maps = []
                            
                            for i in range(1,5):
                                score_cam = ScoreCam(self.model, f"layer{i}")

                                eval_count+=1

                                top_images = []
                                bottom_images = []

                                images = data["image"]
                                names = data["file_name"]

                                for idx, (image,name) in enumerate(zip(images,names),1):

                                    no_trans, heatmap_image = score_cam.generate_cam(input_image=image, filename=name)
                                    
                                    if idx <= int(len(data)):
                                        top_images.append(np.array(heatmap_image))
                                        
                                    else:
                                        bottom_images.append(np.array(heatmap_image))
                                        
                                top_images = np.hstack(top_images)
                                bottom_images = np.hstack(bottom_images)
                                all_images = np.vstack((top_images,bottom_images))
                                
                                all_layers_maps.append(all_images)
                            
                            all_layers_maps = np.vstack(all_layers_maps)
                            
                            self.writer.add_image("Class Activation Maps, Layers 1-4", all_layers_maps, global_step = epoch)
                                
                                
                                
                                
                            
                            
                            

                    val_epoch_loss.append(torch.tensor(val_batch_loss).mean())
                    val_epoch_acc.append(torch.tensor(val_batch_acc).mean())
                    
                    
                    # Visualise the predictions on the last validation batch
                    
                    print(f"len inputs of last batch: {len(inputs)}")
                    
                    self.writer.add_scalar("Validation Loss", 
                                           running_val_loss/len(self.valloader),
                                           global_step = epoch
                                          )
                    
                    self.writer.add_figure("Predictions vs. GT",
                                           self.plot_classes_preds(inputs, labels),
                                           global_step = epoch
                                          )
                    
                    running_val_loss = 0.0
                    
                    
            # Save the model which yielding best acc
            if val_acc > best_val_acc:
                print(f"Saving model at epoch: {epoch}")
                best_val_acc = val_acc
                self.best_model_wts = copy.deepcopy(self.model.state_dict())



            # Print out 

            self.lr_scheduler.step(val_acc)


        self.model = self.model.load_state_dict(self.model.state_dict())
    
    def matplotlib_imshow(self, img, one_channel=False):
        if one_channel:
            img = img.mean(dim=0)
        img = img / 2 + 0.5     # unnormalize
        npimg = img.cpu().numpy()
        if one_channel:
            plt.imshow(npimg, cmap="Greys")
        else:
            plt.imshow(np.transpose(npimg, (1, 2, 0)))
            
    def images_to_probs(self, images):
        '''
        Generates predictions and corresponding probabilities from a trained
        network and a list of images
        '''
        output = self.model(images)
        # convert output probabilities to predicted class
        _, preds_tensor = torch.max(output, 1)
        preds = np.squeeze(preds_tensor.cpu().numpy())
        return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


    def plot_classes_preds(self, images, labels):
        '''
        Generates matplotlib Figure using a trained network, along with images
        and labels from a batch, that shows the network's top prediction along
        with its probability, alongside the actual label, coloring this
        information based on whether the prediction was correct or not.
        Uses the "images_to_probs" function.
        '''
        preds, probs = self.images_to_probs(images)
        # plot the images in the batch, along with predicted and true labels
        fig = plt.figure(figsize=(48, 15))
        for idx in np.arange(4):
            ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
            self.matplotlib_imshow(images[idx], one_channel=False)
            ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
                classes[preds[idx]],
                probs[idx] * 100.0,
                classes[labels[idx]]),
                        color=("green" if preds[idx]==labels[idx].item() else "red"), fontsize=40)
        return fig



In [3]:
"""

Import all data from the database

"""


db_reader = DatabaseReader("crepcheque")

get_images_query ="""

                    SELECT
                        r.crep_id,
                        r.raw_brand_text,
                        r.image_id,
                        r.image_type,
                        r.image_file_path
                    FROM raw_creps r
                    LEFT JOIN
                        images i
                        ON r.crep_id = i.crep_id
                    WHERE 
                        r.images IS NOT NULL
                        AND r.images_processed IS NOT NULL
                        AND i.image_downloaded = true


                    """

database_df = db_reader.send_query(query=get_images_query, return_as_df=True)

In [ ]:
"""

Shuffle the dataframe and split into train and validation

""";

# shuffled_df = sklearn.utils.shuffle(database_df)

# train_df = shuffled_df.iloc[:int(database_df.shape[0]*0.8), :]
# val_df = shuffled_df.iloc[int(database_df.shape[0]*0.8):, :]

X = database_df.image_file_path
y = database_df.raw_brand_text

X_train,y_train, X_valid, y_valid = train_test_split(X,y, test_size = 0.2 ,stratify = y, 
                                                     shuffle = True, random_state = 1994)

train_df = pd.concat([X_train, y_train],axis=1)
val_df = pd.concat([X_valid,y_valid], axis=1)



In [4]:
"""

Define the image transformatoins for the train and validation sets
Define 

"""
train_transforms = train_transformations()
valid_transforms = valid_transformations()

train_set = ShoeDataSet(train_df, transform=train_transforms)
val_set = ShoeDataSet(val_df, transform=valid_transforms)


train_loader = DataLoader(train_set, shuffle=True, batch_size=4)
val_loader = DataLoader(val_set, shuffle=True, batch_size=4)


Compose(
    RandomPerspective(p=0.4)
    RandomHorizontalFlip(p=0.2)
    Resize(size=(300, 300), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [ ]:
"""

Add layers which are to be unfrozen for finetuning purposes

Options: fc (bare minimum), layer1-4 (conv2d layer bottlenecks)

"""

custom_model = custom_model("resnet101",[])

In [ ]:
"""

Redefine the FC layer of your model so that it matches 
the number of classes present in the dataset

Ensure that it is placed on the GPU - .cuda()

"""

in_features = custom_model.model.avgpool.out_features

custom_model.model.fc = torch.nn.Linear(in_features, len(y)).cuda()

# Define the output directory for the logs
custom_model.output_dir = "./logs"



In [ ]:
"""

Define the loss functoin (criterion)
OPtimizer - Adam
LRScheduler

"""

criterion = torch.nn.MSELoss
optimizer = torch.optim.Adam
learning_rate = 0.001
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau

In [ ]:
"""

Initialise the training of the model


"""
# small eval period for debugging purposes

model.train(loss_function = criterion, optimizer = optimizer, lr_scheduler = lr_scheduler, 
           learning_rate = learning_rate, epochs = 10, trainloader = train_loader, 
           val_loader = val_loader, eval_period = 2)
